In [23]:
import glob
import os
import numpy as np
from PIL import Image
from tqdm.auto import tqdm

from monai.transforms import Resize, EnsureChannelFirst, LoadImage, Compose, ScaleIntensity
from monai.data import ArrayDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [24]:
transformer = Compose([LoadImage(image_only=True),
                       EnsureChannelFirst(),
                       ScaleIntensity()])

train_image_path = "data_polar/REFUGE2/Train/Images/"
train_dm_path = "data_polar/REFUGE2/Train/Disc_Masks/"
test_image_path = "data_polar/REFUGE2/Test/Images/"
test_dm_path = "data_polar/REFUGE2/Test/Disc_Masks/"
val_image_path = "data_polar/REFUGE2/Validation/Images/"
val_dm_path = "data_polar/REFUGE2/Validation/Disc_Masks/"

train_data = ArrayDataset(img=sorted([train_image_path + file for file in os.listdir(train_image_path)]),
                          img_transform=transformer,
                          seg=sorted([train_dm_path + file for file in os.listdir(train_dm_path)]),
                          seg_transform=transformer)

train_dataloader = DataLoader(train_data,
                              batch_size=1,
                              shuffle=False,
                              num_workers=2)

val_data = ArrayDataset(img=sorted([val_image_path + file for file in os.listdir(val_image_path)]),
                        img_transform=transformer,
                        seg=sorted([val_dm_path + file for file in os.listdir(val_dm_path)]),
                        seg_transform=transformer)

val_dataloader = DataLoader(val_data,
                            batch_size=1,
                            shuffle=False,
                            num_workers=2)

test_data = ArrayDataset(img=sorted([test_image_path + file for file in os.listdir(test_image_path)]),
                         img_transform=transformer,
                         seg=sorted([test_dm_path + file for file in os.listdir(test_dm_path)]),
                         seg_transform=transformer)

test_dataloader = DataLoader(test_data,
                             batch_size=1,
                             shuffle=False,
                             num_workers=2)

In [26]:
new_path = "data_topunet/REFUGE2/Train/"
old_path = "data_polar/REFUGE2/Train/"
names = sorted(os.listdir(old_path + "Images/"))

if not os.path.exists(new_path + "q_Masks"):
    os.makedirs(new_path + "q_Masks")
    
if not os.path.exists(new_path + "s_Masks"):
    os.makedirs(new_path + "s_Masks")


for j, batch in tqdm(enumerate(train_dataloader)):
    mask = (np.array(255*batch[1][0].permute(2, 1, 0)).astype(np.uint8) > 127).astype(np.uint8)
    diff = np.diff(mask, axis=1)
    mask[:, :, 2] = mask[:, :, 2] - mask[:, :, 1]
    
    new_image = Image.fromarray(mask*255)
    new_image.save(new_path + "Disc_Masks/" + names[j])
    # diff[:, :, 0] = 0
    # 
    # indices_cup = np.argmax(diff[:, :, 1], axis=1)
    # indices_disc = np.argmax(diff[:, :, 2], axis=1)
    # output = np.vstack([indices_cup, indices_disc])
    # np.save(new_path + "s_Masks/" + names[j][:-3] + "npy", output)
    # 
    # q_image = np.zeros(shape=mask.shape).astype(np.uint8)
    # q_image[range(0, mask.shape[0]), list(indices_cup), 1] = 255
    # q_image[range(0, mask.shape[0]), list(indices_disc), 2] = 255
    # q_image = Image.fromarray(q_image)
    # plt.imshow(q_image)
    # q_image.save(new_path + "q_Masks/" + names[j])
    

0it [00:00, ?it/s]